In [2]:
import tensorflow as tf


from imblearn.over_sampling import SMOTE
from collections import Counter
from numpy import * 
import os

import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
def dataLoadSm():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    print (y_val)
    
    print('Before Smote')

    dff = y_train
    M = y_train.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
    
    
    smt = SMOTE()
    
    X_train_sm, Y_train_sm = smt.fit_resample(x_train, y_train)
    
    print('After SMOTE')
    dff = Y_train_sm
    M = Y_train_sm.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
        
    return X_train_sm, Y_train_sm, x_val, y_val, x_test, y_test

def dataLoad():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    #print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    #print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    #print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    #print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    #print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    #print (y_val)
    
        
    return x_train, y_train, x_val, y_val, x_test, y_test

In [4]:
x_train_sm, y_train_sm, x_val, y_val, x_test, y_test = dataLoadSm()
x_train, y_train, x_val, y_val, x_test, y_test = dataLoad()

   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
******training features******
               vo          r      u_200     u_850      v_200     v_85

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#scaled_x_training = scaler.fit_transform(x_training)
#df = pd.DataFrame(StandardScaler().fit_transform(x_training))

X_train_stand = x_train_sm.copy() #smote
X_valid_stand = x_val.copy() #smote
X_test_stand = x_test.copy() #smote

#not smote
X_train_scaled = x_train.copy() 
X_valid_scaled = x_val.copy()
X_test_scaled = x_test.copy()

num_cols = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the training data column
    X_valid_stand[i] = scale.transform(X_valid_stand[[i]])   

    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])
    
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_scaled[[i]])
    
    # transform the training data column
    X_train_scaled[i] = scale.transform(X_train_scaled[[i]])
    
    # transform the training data column
    X_valid_scaled[i] = scale.transform(X_valid_scaled[[i]])   

    # transform the testing data column
    X_test_scaled[i] = scale.transform(X_test_scaled [[i]])

In [13]:
# explore adaboost ensemble number of trees effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from matplotlib import pyplot

# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    #n_trees = [10, 50, 100, 500, 1000, 5000]
    #learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0, 2.0]
    n_trees = [10, 50]
    learning_rates = [0.001]
    for n in n_trees:
        for l in learning_rates:
            models[str(n)+'trees_'+str(l)+'learning_rate'] = AdaBoostClassifier(n_estimators=n, learning_rate=l) 
    
    return modelsQuick  


# evaluate a given model using cross-validation
def evaluate_model(model, x_train, y_train, x_val, y_val):
    trained_mod = model.fit(x_train, y_train)
    # define the evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    print("everything is ok!")
    # evaluate the model and collect the results
    scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv)
    conf = confusion_matrix(y_val, np.round(model.predict(x_val)))
    return scores


In [14]:
models = get_models()

In [20]:
print("#------------#--------------- Smote is doing ---------------#------------#")

results1, names1, confs1 = list(), list(), list()
for name1, model in models.items():
    res1 = evaluate_model(model, X_train_stand, y_train_sm.values.reshape(-1,), X_valid_scaled, y_val.values.reshape(-1,))
    print(name1)
    print(res1[0])
    print(res1[1])
    results1.append(res1[0])
    names1.append(name)
    confs1.append(res1[1])
    
   
print("the smote is done ---->")

import pickle
sample_list = results1
file_name = "results_adabost_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the results for smote is saved on results_adabost_smote.pkl")

sample_list = names1
file_name = "names_adabost_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the name for smote is saved on names_adabost_smote.pkl")


sample_list = confs1
file_name = "confs_adabost_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the confs for smote is saved on confs_adabost_smote.pkl")



#------------#--------------- Smote is doing ---------------#------------#


KeyboardInterrupt: 